# Augmentation(dog_and_cat)
- https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
- https://www.kaggle.com/c/dogs-vs-cats/data
- 학습 데이터로 1,000장의 고양이 사진과 1,000장의 강아지 사진을 사용 (kaggle  25,000자)
- 검증 데이터로는 각각 400장 사용

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
import tensorflow as tf
import numpy as np
from numpy  import expand_dims
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
np.random.seed(15)

Using TensorFlow backend.


# 1. Augmentation 했을 때

## 1.1 data Augmentation

In [5]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음(이미지 원본을 사용)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        './smallcatdog/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = validation_datagen.flow_from_directory(
        './smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


## 1. 2. model

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## 1.3. train

In [7]:
# steps_per_epoch는 한 세대마다 몇 번 생성기로부터 데이터를 얻을지를 나타내는 값
# 한 세대마다 사용되는 학습데이터의 수는 steps_per_epoch * batch_size
        
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,    # 2000/16     한번에 125개씩 생성
        epochs=5,  #50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)     # 800/16   한번에 50개씩 생성


Epoch 1/5
125/125 [==============================] - 21s 170ms/step - loss: 0.7238 - accuracy: 0.5575 - val_loss: 0.6698 - val_accuracy: 0.5738
Epoch 2/5
125/125 [==============================] - 19s 148ms/step - loss: 0.6556 - accuracy: 0.6215 - val_loss: 0.5199 - val_accuracy: 0.6037
Epoch 3/5
125/125 [==============================] - 20s 158ms/step - loss: 0.6242 - accuracy: 0.6655 - val_loss: 0.5850 - val_accuracy: 0.6850
Epoch 4/5
125/125 [==============================] - 19s 153ms/step - loss: 0.5999 - accuracy: 0.6975 - val_loss: 0.4630 - val_accuracy: 0.7287
Epoch 5/5
125/125 [==============================] - 19s 155ms/step - loss: 0.5661 - accuracy: 0.7195 - val_loss: 0.9516 - val_accuracy: 0.5925


## 1. 4. evaluate

In [8]:
# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator( test_generator,       steps = 5)
print(scores[1])

-- Evaluate --
0.574999988079071


# 2.augmentation 없이  학습

In [11]:
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255 )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'smallcatdog/train',  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50, # 50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

# 구조, weight 모두 저장하기
model.save("smallcatdog.h5")
scores = model.evaluate_generator( test_generator,       steps = 5)
print(scores[1])

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 18s 142ms/step - loss: 0.7339 - accuracy: 0.5150 - val_loss: 0.6982 - val_accuracy: 0.5412
Epoch 2/50
125/125 [==============================] - 17s 139ms/step - loss: 0.6759 - accuracy: 0.5995 - val_loss: 0.6277 - val_accuracy: 0.5800
Epoch 3/50
125/125 [==============================] - 18s 145ms/step - loss: 0.6240 - accuracy: 0.6645 - val_loss: 0.6527 - val_accuracy: 0.6988
Epoch 4/50
125/125 [==============================] - 18s 141ms/step - loss: 0.5729 - accuracy: 0.7165 - val_loss: 0.5978 - val_accuracy: 0.6950
Epoch 5/50
125/125 [==============================] - 18s 142ms/step - loss: 0.5078 - accuracy: 0.7755 - val_loss: 0.6428 - val_accuracy: 0.6875
Epoch 6/50
125/125 [==============================] - 18s 145ms/step - loss: 0.4626 - accuracy: 0.7825 - val_loss: 0.8733 - val_accuracy: 0.7200
Epoch 

> 인공지능이 고 정밀도 실수 연산이 필요할까?
- 비트 수(부동소스점) 연산에 이상이 있어도
- 반복하다보니, 정밀도는 개선이 되고
- 정밀도가 조금 떨어지더라도 고속 연산이 더 중요

> GPU
- 2000개 vs 1000 2개
- 1000개짜기 가격이 더 싸고, 2개 사이에 연산이 이뤄지므로
- 2000개는 빠르고, 그 안에서 모든 계산이 이뤄지기 때문에,
- 여러개 gpu간의 병렬처리가 현실적

# 3. model save and load

## 3.1. model architectrue, weight 모두 저장

In [5]:
model2.save('bicycleship.h5')

In [14]:
from keras.models import load_model
model2 = load_model('smallcatdog.h5')
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)       

In [15]:
batch_size = 16
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

scores = model2.evaluate_generator(test_generator, steps = 800//16)
print(scores[0])

Found 800 images belonging to 2 classes.
0.2811466455459595


In [2]:
img_path = 'imagenet/ship.jpg'
img = load_img(img_path, target_size=(32, 32))
x = img_to_array(img)
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)
preds = model2.predict(x)

print(preds)
print('Probability that the image is a Bicycle:', preds[0,0])
print('Probability that the image is a Ship:', preds[0,1])

(32, 32, 3)
(1, 32, 32, 3)
[[1.0669018e-10 1.0000000e+00]]
Probability that the image is a Bicycle: 1.0669018e-10
Probability that the image is a Ship: 1.0
